In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1606962788567_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql import SQLContext
from pyspark.sql.functions import col, avg, mean, when, count, isnan
from pyspark import SparkContext
import numpy as np
from numpy.linalg import eigh

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [133]:
# load dataset
df = spark.read.format('csv').options(header='true', inferSchema='true').load("s3://afung-10605-project/songs/data/b*")

df.select('artist_familiarity').show()
df.select('song_hotttnesss').show()
df.select('analysis_sample_rate').show()
df.select('danceability').show()
df.select('duration').show()
df.select('end_of_fade_in').show()
df.select('energy').show()
df.select('key').show()
df.select('loudness').show()
df.select('mode').show()
df.select('start_of_fade_out').show()
df.select('tempo').show()
df.select('year').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+
|artist_familiarity|
+------------------+
|0.6652990520943541|
| 0.690885497114412|
|0.5940491245011125|
| 0.833591719824811|
|0.7380844177161777|
|               0.0|
|0.5594202480844251|
|0.5244855494695024|
|0.8831436650054394|
|0.4426956666168511|
|0.5893312546494746|
|0.3617999581550273|
|0.5228802505435669|
|0.6745516134186544|
|0.6458308589757341|
|0.6883056226085532|
|0.4922427785820167|
| 0.560868629507006|
|0.6567001580510525|
|0.8038153696948831|
+------------------+
only showing top 20 rows

+-------------------+
|    song_hotttnesss|
+-------------------+
| 0.6980975442614767|
| 0.4909042005420652|
|               null|
|0.24686509445288138|
| 0.7271405818437788|
| 0.7389245482640411|
|0.38393468427870303|
|                0.0|
| 0.5755105376219763|
|                0.0|
|                0.0|
|                0.0|
|               null|
| 0.6141845069522139|
| 0.6130458357683207|
|0.42758288620768037|
|0.26586104921065007|
|               null|
| 0.3775

In [136]:
df.select('danceability').distinct().show()
df.select('analysis_sample_rate').distinct().show()
df.select('energy').distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------+
|danceability|
+------------+
|         0.0|
+------------+

+--------------------+
|analysis_sample_rate|
+--------------------+
|               22050|
+--------------------+

+------+
|energy|
+------+
|   0.0|
+------+

In [137]:
# drop features that have only one value, or are unneeded like _c0 or song_id
sample = df.drop('song_id', '_c0', 'danceability', 'energy',
                          'analysis_sample_rate')
# drop all the rows with null values
sample = sample.na.drop()
sample.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

286453

In [138]:
# column names
sample.printSchema()

sample.show(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_familiarity: double (nullable = true)
 |-- artist_hotttnesss: double (nullable = true)
 |-- song_hotttnesss: double (nullable = true)
 |-- duration: double (nullable = true)
 |-- end_of_fade_in: double (nullable = true)
 |-- key: integer (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: integer (nullable = true)
 |-- start_of_fade_out: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- year: integer (nullable = true)

+------------------+-------------------+-------------------+---------+--------------+---+--------+----+-----------------+-------+----+
|artist_familiarity|  artist_hotttnesss|    song_hotttnesss| duration|end_of_fade_in|key|loudness|mode|start_of_fade_out|  tempo|year|
+------------------+-------------------+-------------------+---------+--------------+---+--------+----+-----------------+-------+----+
|0.6652990520943541|0.40107322089442643| 0.6980975442614767|285.02159|         0.136|  7|  -7.363|   1|           275.04|

In [139]:
# take out just song_hotttnesss because that's the predictive variable
sample_labels = sample.select('song_hotttnesss')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [140]:
sample_labels.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|    song_hotttnesss|
+-------------------+
| 0.6980975442614767|
| 0.4909042005420652|
|0.24686509445288138|
| 0.7271405818437788|
| 0.7389245482640411|
|0.38393468427870303|
|                0.0|
| 0.5755105376219763|
|                0.0|
|                0.0|
|                0.0|
| 0.6141845069522139|
| 0.6130458357683207|
|0.42758288620768037|
|0.26586104921065007|
| 0.3775316449767025|
| 0.5647575777499548|
| 0.2707759989463275|
| 0.6450385130033194|
|                0.0|
+-------------------+
only showing top 20 rows

In [141]:
# check if there are missing values in each of the columns -> there shouldn't be any because we dropped all the rows with null values at the beginning
sample.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in sample.columns]).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-----------------+---------------+--------+--------------+---+--------+----+-----------------+-----+----+
|artist_familiarity|artist_hotttnesss|song_hotttnesss|duration|end_of_fade_in|key|loudness|mode|start_of_fade_out|tempo|year|
+------------------+-----------------+---------------+--------+--------------+---+--------+----+-----------------+-----+----+
|                 0|                0|              0|       0|             0|  0|       0|   0|                0|    0|   0|
+------------------+-----------------+---------------+--------+--------------+---+--------+----+-----------------+-----+----+

In [62]:
# this function calculates the average of the column
def takeAvg(dataFrame, column):
    return dataFrame.select(mean(col(column))).collect()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [63]:
# fill in the na values and return a new data frame
# if the value is taken from takeAvg we need to index [0][0] in order to get subValue
def fillNa(dataFrame, subDict):
    return dataFrame.fillna(subDict)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [142]:
# example: fill null's in song_hotttness with average
# but not actaully used because we already dropped the rows with null song_hotttnesss
song_hotAvg = takeAvg(sample, 'song_hotttnesss')[0][0]
sample_data = fillNa(sample, {'song_hotttnesss':song_hotAvg})

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [143]:
# we drop song_id becuase it's a string and we don't need it in our analysis
# and we drop song_hotttnesss becuase it is the predictive variable
sample_data = sample.drop('song_hotttnesss')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [144]:
sample_data.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist_familiarity: double (nullable = true)
 |-- artist_hotttnesss: double (nullable = true)
 |-- duration: double (nullable = true)
 |-- end_of_fade_in: double (nullable = true)
 |-- key: integer (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: integer (nullable = true)
 |-- start_of_fade_out: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- year: integer (nullable = true)

In [145]:
# this function centers each column
def centerData(dataFrame_rdd):
    col_mean = dataFrame_rdd.map(np.array).sum()/dataFrame_rdd.count()
    return dataFrame_rdd.map(lambda x: x-col_mean)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [146]:
# find the covariance matrix of the dataframe
def findCov(dataFrame_rdd): 
    centered = centerData(dataFrame_rdd)
    return (centered.map(lambda x: np.outer(x,x)).reduce(lambda x, y : x+y))/centered.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [147]:
# eigen decomposition
def pca(dataFrame_rdd, k = 5):
    cov = findCov(dataFrame_rdd)
    eig_vals, eig_vecs = eigh(cov)
    eig_vals_indices = np.argsort(eig_vals)[::-1][0:k]
    components = np.take(eig_vecs, eig_vals_indices, axis = 1)
    eigenvalues = eig_vals[np.argsort(eig_vals)[::-1]]
    scores = dataFrame_rdd.map(lambda x: np.dot(x, components))
    return (components, scores, eigenvalues)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [148]:
# create rdd from the dataframe
sample_rdd = sample_data.rdd

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [149]:
components, scores, eigenvalues = pca(sample_rdd)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [150]:
components

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([[-3.51003663e-05,  7.22573778e-06, -6.96848672e-05,
        -5.43854968e-04, -3.67943051e-03],
       [-2.91319661e-05, -6.74507972e-08, -1.63053628e-05,
        -2.77146407e-04, -2.00809232e-03],
       [ 1.79891926e-03, -7.13814217e-01,  2.30900822e-03,
         6.84379664e-01, -1.47564662e-01],
       [-5.56084653e-05, -1.96646186e-03,  2.21091854e-03,
         4.24051737e-02,  8.75575682e-02],
       [ 6.03749949e-06, -4.18662728e-04, -1.70846919e-03,
        -1.01318058e-02, -4.92088839e-02],
       [-3.30600154e-04,  5.06606252e-04, -2.34112381e-02,
        -2.06640799e-01, -9.72249390e-01],
       [ 2.51127939e-06,  1.96541649e-04,  1.00618062e-04,
         6.76918848e-04,  2.15698516e-03],
       [ 1.99756414e-03, -7.00302983e-01, -1.06327153e-02,
        -6.97730384e-01,  1.49665541e-01],
       [-6.87650043e-04,  5.78257548e-03, -9.99662566e-01,
         1.39525345e-02,  2.11149959e-02],
       [-9.99996093e-01, -2.68704158e-03,  6.77944516e-04,
        -1.06287226e-04

In [151]:
scores.take(3)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[array([-2008.01754045,  -400.71224298,  -131.53140106,     6.22766674,
           9.3663414 ]), array([-2.00754349e+03, -5.44035332e+02, -1.60322158e+00, -1.76732537e+00,
        9.11026738e+00]), array([   0.34208762, -155.22555178, -112.75512788,    6.122216  ,
          5.72407236])]

In [152]:
eigenvalues

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([9.62905071e+05, 2.74872380e+04, 1.21931217e+03, 3.85236157e+01,
       2.33545258e+01, 1.29006864e+01, 1.00071436e+01, 2.13424273e-01,
       2.31655182e-02, 4.00318562e-03])

**Linear Regression**

In [154]:
len(sample_labels.rdd.collect())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

286453

In [155]:

scores_np = np.array(scores.collect())
inv_outer = np.linalg.inv(scores.map(lambda pt: np.outer(pt, pt)).sum())
scores_w = np.matmul(np.matmul(inv_outer, np.transpose(scores_np)), np.array(sample_labels.rdd.collect()))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [156]:
scores_w

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([[-0.00010615],
       [-0.00013282],
       [-0.00136346],
       [ 0.00132717],
       [-0.00066829]])

In [157]:
np.matmul(components, scores_w)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([[ 1.83491121e-06],
       [ 9.99497370e-07],
       [ 1.09837710e-03],
       [-4.98216589e-06],
       [ 2.18234844e-05],
       [ 4.07383401e-04],
       [-7.06661349e-07],
       [-9.18726694e-04],
       [ 1.36671194e-03],
       [ 1.05218785e-04]])